In [1]:
from IPython.display import display
import pandas as pd
import os
import csv
from PIL import Image
# from my_yolo import *

images = os.listdir(r"C:\Users\BMEi\Documents\GitHub\WORK\Windows\CODE_BME\PROJECT_MALARIA\malaria\images") # read the images data where all images are present
train = pd.read_json(r"C:\Users\BMEi\Documents\GitHub\WORK\Windows\CODE_BME\PROJECT_MALARIA\malaria\training.json") # read the train json file where info about all images
# are given including bounding box information.
test = pd.read_json(r"C:\Users\BMEi\Documents\GitHub\WORK\Windows\CODE_BME\PROJECT_MALARIA\malaria\test.json") # read the test json file.

# here we have store train and test image in different list
train_image = [] # train images stored here
test_image = [] # test images stored here
train_data = []
test_data = []

for index, row in train.iterrows():
    image_name = row['image']['pathname'].split('/')[-1]
    label = row['objects'][0]['category'] if row['objects'] else 'unknown'
    for obj in row['objects']:
        bbox = obj['bounding_box']
        train_data.append([image_name, label, bbox['minimum']['c'], bbox['minimum']['r'], bbox['maximum']['c'], bbox['maximum']['r']])
    train_image.append(image_name)

for index, row in test.iterrows():
    image_name = row['image']['pathname'].split('/')[-1]
    label = row['objects'][0]['category'] if row['objects'] else 'unknown'
    for obj in row['objects']:
        bbox = obj['bounding_box']
        test_data.append([image_name, label, bbox['minimum']['c'], bbox['minimum']['r'], bbox['maximum']['c'], bbox['maximum']['r']])
    test_image.append(image_name)

print("Train image length is -> ", len(train_image))
print("Train image :", train_image[:5])
print("Test image length is -> ", len(test_image))
print("Test image :", test_image[:5])

class_dict = {'red blood cell':0,'trophozoite': 1, 'schizont': 2, 'difficult': 3, 'ring': 4,'leukocyte': 5, 'gametocyte': 6}

# Define output directories
train_output_dir = r"C:\Users\BMEi\Documents\GitHub\WORK\Windows\CODE_BME\PROJECT_MALARIA\DATA_SET\train_images"
test_output_dir = r"C:\Users\BMEi\Documents\GitHub\WORK\Windows\CODE_BME\PROJECT_MALARIA\DATA_SET\test_images"
yolo_output_dir = r"C:\Users\BMEi\Documents\GitHub\WORK\Windows\CODE_BME\PROJECT_MALARIA\DATA_SET"

# Create directories if they don't exist
os.makedirs(train_output_dir, exist_ok=True)
os.makedirs(test_output_dir, exist_ok=True)
os.makedirs(yolo_output_dir, exist_ok=True)

# Save train images
for image_name in train_image:
    src_path = os.path.join(r"C:\Users\BMEi\Documents\GitHub\WORK\Windows\CODE_BME\PROJECT_MALARIA\malaria\images", image_name)
    dest_path = os.path.join(train_output_dir, image_name)
    if os.path.exists(src_path):
        Image.open(src_path).save(dest_path)

# Save test images
for image_name in test_image:
    src_path = os.path.join(r"C:\Users\BMEi\Documents\GitHub\WORK\Windows\CODE_BME\PROJECT_MALARIA\malaria\images", image_name)
    dest_path = os.path.join(test_output_dir, image_name)
    if os.path.exists(src_path):
        Image.open(src_path).save(dest_path)

# Save train.csv
train_csv_path = os.path.join(yolo_output_dir, "train.csv")
with open(train_csv_path, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["image_name", "label", "xmin", "ymin", "xmax", "ymax"])
    writer.writerows(train_data)

# Save test.csv
test_csv_path = os.path.join(yolo_output_dir, "test.csv")
with open(test_csv_path, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["image_name", "label", "xmin", "ymin", "xmax", "ymax"])
    writer.writerows(test_data)

print(f"CSV files saved at: {yolo_output_dir}")


Train image length is ->  1208
Train image : ['8d02117d-6c71-4e47-b50a-6cc8d5eb1d55.png', '10be6380-cbbb-4886-8b9e-ff56b1710576.png', '6b14c855-8561-417c-97a4-63fa552842fd.png', '13099edb-35d9-438f-b093-2cf2ebf9d255.png', '2559636b-f01a-4414-93da-210c3b12d153.png']
Test image length is ->  120
Test image : ['41be1bd3-0d31-4881-bf1f-3ccdfa21ff12.jpg', '64985a1e-16bb-4016-a01c-c21a5b86e572.jpg', 'c70894fa-50e8-4eed-a5a8-159a20153a49.jpg', 'abb72ce2-4193-4dd3-a34c-61c2ee22ccff.jpg', '8db76867-671a-4488-a7b6-991e4dd2e05f.jpg']
CSV files saved at: C:\Users\BMEi\Documents\GitHub\WORK\Windows\CODE_BME\PROJECT_MALARIA\DATA_SET
